In [ ]:
# default_exp params
# hide
_FNAME='params'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
from tempfile import TemporaryDirectory

TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')

In [ ]:
#export
import collections
import yaml

from dvcrecord.utils import PARAMS_FILE_DEFAULT, maybe_yaml, write_yaml

class ParamFile:
    def __init__(self, fpath=None):
        self._accessed = collections.OrderedDict()

        self.fpath = fpath or PARAMS_FILE_DEFAULT
        #no support for .py params just yet.  Could pick up from the file extension
        #or just reuse the logic directly from the dvc library
        with open(self.fpath) as f:
            self._file_contents = yaml.safe_load(f)

    def load(self, name):
        try:
            return self._accessed[name]
        except KeyError:
            returnme = self._file_contents
            for key in name.split('.'):
                returnme = returnme[key]

            self._accessed[name] = returnme
            return returnme

    def render(self, as_yaml=False):
        keys = list(self._accessed.keys())
        returnme = {self.fpath: keys}
        return maybe_yaml(returnme, as_yaml=as_yaml)

In [ ]:
def test_paramfile():
    with TemporaryDirectory() as tempdir:
        params = {'myval': 1, 'stagename': {'otherval': 2}}

        fpath = write_yaml(params, folder=tempdir, fname='params.yaml')
        p = ParamFile(fpath)
        TESTCASE.assertEqual(p.load('myval'), 1)
        TESTCASE.assertEqual(p.load('stagename.otherval'), 2)

        TESTCASE.assertEqual(p.render(as_yaml=False), {os.path.join(tempdir, 'params.yaml'): ['myval', 'stagename.otherval']})
test_paramfile()

In [ ]:
#export
class Params:
    def __init__(self, param_files=None):
        '''
        param_files: Iterable of ParamFile
        '''
        self._params = collections.OrderedDict({})
        param_files = param_files or []
        [self.register(param_file) for param_file in param_files]

    def render(self, as_yaml=False):
        params_config = []
        for name, param in self._params.items():
            params_config.extend([param.render(as_yaml=False)])
        return maybe_yaml(params_config, as_yaml=as_yaml)

    def register(self, param, name=None):
        name = name or param.fpath
        self._params[name] = param
        
    def list_of_files(self):
        return [fpath for fpath in self._params]
        
    def load(self, path):
        '''
        Lazily access parameters in files
        filename:paramname
        e.g. my/dir/params.yaml: epochs
        '''
        sep = ':'
        if sep in path:
            fname, param_name = path.split(sep)
        else:
            fname = PARAMS_FILE_DEFAULT
            param_name = path

        fname = fname.strip()
        param_name = param_name.strip()

        try:
            pf = self._params[fname]
        except KeyError:
            pf = ParamFile(fpath=fname)
            self.register(pf)

        return pf.load(param_name)


In [ ]:

def test_params():
    with TemporaryDirectory() as tempdir:
        param1 = {'one': 1}
        param2 = {'two': 2}

        onepath = write_yaml(param1, folder=tempdir, fname='one.yaml')
        twopath = write_yaml(param2, folder=tempdir, fname='two.yaml')

        p = Params()
        TESTCASE.assertEqual(p.load(onepath+':one'), 1)
        TESTCASE.assertEqual(p.load(twopath+':two'), 2)

        TESTCASE.assertEqual(p.render(as_yaml=False),
                [
                    {onepath:
                            ['one']},
                    {twopath:
                            ['two']}
                ])
        TESTCASE.assertEqual(p.list_of_files(), [onepath, twopath])
test_params()

In [ ]:
notebook2script(_nbpath)

Converted params.ipynb.
